# Eval

In [ ]:
#| default_exp eval

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from stringdale import (
    Define,
    Scope,
    V,
    E,
    Condition,
    draw_nx
)

from frozendict import frozendict
from stringdale.core import  checkLogs
import pytest
import asyncio
from pydantic import BaseModel, ConfigDict

## Using podtw

In [ ]:
# use it on the example files

## Running and evaluating a single data point

In [ ]:
# define datapoint class


# take react and eval it.
# build cosine and chat ranking function as example
# also use the regex

In [ ]:
# get a dataSeries the distance per step and total metrics such as sum of distances and coverage

## DataSets

In [ ]:
# lift this to a least of datapoints
# make the dataset class
# get it from a directory, and then eval it.

# make the time series into a dataframe of all datapoints (in case of directory take the filename relative to the directory)
# maybe give ability to have nested directories with regexs to filter the files?
 
# then we output a datasetRun class that has the results dataframe but also metadata about the run
# for now lets start with an agent name and run id

## Comparing Runs

In [ ]:
# a function that takes 2 datasetRuns and returns a comparison per datapoint on the difference between the two runs
# then have a utility function that prints the summary
# and have a utility function that returns the k datapoints that regressed the most
# have a pprint version of it that actually plots the traces and the difference between them



## End to End regression testing

In [ ]:
## show the following example

# always have a factory (or a nested factory expression)
# take the same factory and change the prompt
# take the same agent but give it a different db
# or maybe the same db but a different filter
# same factory but different fewshot examples yaml files
# and show regression testing

# our function should take 2 agents, and a EvalDataSet 
# return the comparison and print the summary as the k datapoints that regressed the most



## training and test set

In [ ]:
## TODO in future, specify 2 datasets (by dirs and regex)
# one is the train one is the test
# we take an agent (or 2 for comparison)
# and we do the same logic for evaluating and comparing, however, we print the statistics only for the test set
# but we show the regression for the train set.

## Design

In [ ]:
"""

Data model

We have a dataset
* containing tests
* each test has the input to the agent
* and the expected output
* test is any object that can be serialized to json
* expected output is a partial trace spec

* partial trace spec is a list of steps
* each step has a name is a dict with accessors and value are how to check them
* names are the node name we expect to see in the trace
* the dict defines what we expect the value to look like


When we run a dataset, we take the input, run the agent, and check the output against the partial trace spec
since the partial trace spec does not 


"""

'\n\nData model\n\nWe have a dataset\n* containing tests\n* each test has the input to the agent\n* and the expected output\n* test is any object that can be serialized to json\n* expected output is a partial trace spec\n\n* partial trace spec is a list of steps\n* each step has a name is a dict with accessors and value are how to check them\n* names are the node name we expect to see in the trace\n* the dict defines what we expect the value to look like\n\n\nWhen we run a dataset, we take the input, run the agent, and check the output against the partial trace spec\nsince the partial trace spec does not \n\n\n'

In [ ]:
class ExpectedTrace:
    pass

class DataPointRun:
    # basically a list of traces, agent input and agent output
    pass




In [ ]:
def collect_traces_from_file(file_path):
    pass

def collect_traces_from_logg_aggregator(logger):
    pass

def run_dataset(agent,dataset,output_dir):
    # for each data point in the dataset
    # run the agent
    # collect the traces into a file
    # return the file path
    pass

def write_comparison_to_file(dataset_run,expected_traces,output_dir):
    # run the comparison and write the results to a file
    pass


In [ ]:
def runs_summary(runs,dir):
    # get the run files and the comparison files
    # get the total metrics per expected node and total
    # make them into a dataframe
    pass

def plot_runs(runs,dir):
    # call runs_summary
    # plot the results
    pass

def check_regressions(runs,dir):
    # get two runs
    # for each input, if the second run is worse than the first, then flag it
    # make a dataframe of the regressions on a whole run basis
    
    # also make a dataframe of the regressions on a per node basis for the runs that regressed.
    pass


In [ ]:
class DataSet:
    pass




## Experiment runs

In [ ]:
# TODO start with directories of files with traces.
# here we just run the agent on the input and collect the traces to files
# Later, add a way to customize the runs from a logger or something 
# I think the best way would be to be able to turn the logs into a dataset file and work on it locally.

## Experiment scoring

In [ ]:
# here we use the DPTW to match each trace to an expected trace
# than we have multiple scores
    # total distance, 
    # total distance per expected trace, 
    # coverage (percent of nodes expected), 
    # time coverage (percent of time of nodes expected), used to ignore nodes with no logic


# this experiment object can be dumped into a directory


## Regression detection

In [ ]:

## Regression detection
# here we just compare the runs to each other


## export

In [ ]:
# |hide
import nbdev; nbdev.nbdev_export()